In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

# Packaging code from dense_prototype into a class to instantiate and train based on DNA

In [2]:
DNA_list = [
    [["D",1024],["D",512],["D",128]],
    [["D",1024],["R",0.2],["D",512],["R",0.2],["D",128]]
]

Defaults for the training configurations

In [3]:
batch_size = 16
use_mps = False and torch.backends.mps.is_available()
use_cuda = True and torch.cuda.is_available()
test_batch_size = 128
epochs = 14
lr = 1.0
gamma = 0.7
seed = 1
log_interval = 10
save_model = False

if use_cuda:
    device = torch.device("cuda")
elif use_mps:
    device = torch.device("mps")
else:
    device = torch.device("cpu")
    
train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}

transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)),
        transforms.Lambda(lambda x: torch.flatten(x))
        ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                       transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

NameError: name 'torch' is not defined

In [4]:
class ConstructNet(nn.Module):
    def __init__(self, DNA, input_size=28*28, output_size=10):
        super(ConstructNet, self).__init__()
        self.DNA = DNA
        self.input_size = input_size
        self.output_size = output_size
        self.layers = []
        
        # Append first layer
        self.layers.append(nn.Linear(self.input_size, self.DNA[0][1]))
        self.layers.append(nn.ReLU())
        
        for i in range(1, len(self.DNA)):
            if self.DNA[i][0] == "D":
                # The input size is the output of the last layer
                tmp_input_size = self.last_layer_output_size()
                self.layers.append(nn.Linear(tmp_input_size, self.DNA[i][1]))
                self.layers.append(nn.ReLU())
            if self.DNA[i][0] == "R":
                self.layers.append(nn.Dropout(self.DNA[i][1]))
        
        # Append the output layer        
        self.layers.append(nn.Linear(self.layers[-2].out_features, self.output_size))
        self.layers.append(nn.LogSoftmax(dim=1))
        self.net = nn.Sequential(*self.layers)
        
    def forward(self, x):
        return self.net(x)
    '''
    Based on the layers created, find the output size of the last dense layer
    '''
    def last_layer_output_size(self):
        for layer in self.layers[::-1]:
            if isinstance(layer, nn.Linear):
                return layer.out_features
            
    def train_net(self, device, train_loader, optimizer, epoch, log_interval, print_stats):
        self.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = self.forward(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % log_interval == 0 and print_stats:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))


    def test(self, device, test_loader):
        self.eval()
        test_loss = 0
        correct = 0
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = self.forward(data)
                test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
                pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
                correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(test_loader.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))
        
    def count_parameters(self):
        # https://discuss.pytorch.org/t/how-do-i-check-the-number-of-parameters-of-a-model/4325/7
        return sum(p.numel() for p in self.net.parameters() if p.requires_grad)        
        
    

NameError: name 'nn' is not defined

In [1]:
for DNA in DNA_list:
    inv_net = ConstructNet(DNA)
    print(inv_net)
    optimizer = optim.Adadelta(inv_net.parameters(), lr=lr)
    scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

    for epoch in range(1, epochs + 1):
        inv_net.train_net(device, train_loader, optimizer, epoch, log_interval, print_stats=False)
        inv_net.test(device, test_loader)
        scheduler.step()

    if save_model:
        torch.save(inv_net.state_dict(), "mnist_cnn.pt")

NameError: name 'DNA_list' is not defined